In [1]:
import nilearn
import nibabel as nib
from nilearn import datasets, image, input_data

import sys
import pandas as pd

import os
import glob

from pathlib import Path

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [2]:
# Load Ng images from neurovault
# Read Keren image
# _neg refers to a negative contrast value of MDD - HV, so MDD < HV
# _pos refers to MDD > HV

from nilearn.datasets import fetch_neurovault_ids
ng = fetch_neurovault_ids(collection_ids=[3884]) 

image.load_img(ng['images'][3]).to_filename('../great_expectations_data/previous_metaanalysis_data/ng_neg.nii.gz')
image.load_img(ng['images'][5]).to_filename('../great_expectations_data/previous_metaanalysis_data/ng_pos.nii.gz')

keren_neg = image.load_img('../great_expectations_data/previous_metaanalysis_data/ALEtxt_feedback_MDD+HR+ContvsHV_final_woROI_wsmoski2011_wJon_ALE_C05_1_mni152_thresh.nii.gz')

Reading local neurovault data.
Already fetched 1 image
Already fetched 2 images
Already fetched 3 images
Already fetched 4 images
Already fetched 5 images
Already fetched 6 images
Already fetched 7 images
Already fetched 8 images
Already fetched 9 images
9 images found on local disk.


In [3]:
# Read Zhang coordinates
zhang = pd.read_csv('../great_expectations_data/previous_metaanalysis_data/zhang_coords.csv')
zhang_neg = zhang.loc[zhang['increased/decreased'] == 'decreased']
zhang_pos = zhang.loc[zhang['increased/decreased'] == 'increased']

In [4]:
# Create blank brain images
! module load afni; 3dcalc -a ../great_expectations_data/previous_metaanalysis_data/TT_N27+tlrc.HEAD -expr 'a * 0' -prefix  ../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz -overwrite
! module load afni; 3dcalc -a ../great_expectations_data/previous_metaanalysis_data/TT_N27+tlrc.HEAD -expr 'a * 0' -prefix  ../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz -overwrite

/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
*+ WARNING: output sub-brick 0 is all zeros!
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz
/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
*+ WARNING: output sub-brick 0 is all zeros!
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz


In [5]:
# Project Zhang coordinates onto brain images
for _,row in zhang_pos.iterrows():
    program = 'module load afni; 3dcalc'
    temp = '../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz'
    rsquar = (((float(row.volume)*3.0)/(3.14*4.)) ** (1.0/3.0)) ** 2
    expr = f'step(a+step({rsquar}-(x-{-1*row.x})*(x-{-1*row.x})-(y-{-1*row.y})*(y-{-1*row.y})-(z-{row.z})*(z-{row.z})))'
    prefix = f'../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz'
    runline = f"{program} -a {temp} -expr \"{expr}\" -prefix {prefix} -overwrite"
    if row.volume > 0:
        ! $runline
        
for _,row in zhang_neg.iterrows():
    program = 'module load afni; 3dcalc'
    temp = '../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz'
    rsquar = (((float(row.volume)*3.0)/(3.14*4.)) ** (1.0/3.0)) ** 2
    expr = f'step(a+step({rsquar}-(x-{-1*row.x})*(x-{-1*row.x})-(y-{-1*row.y})*(y-{-1*row.y})-(z-{row.z})*(z-{row.z})))'
    prefix = f'../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz'
    runline = f"{program} -a {temp} -expr \"{expr}\" -prefix {prefix} -overwrite"
    if row.volume > 0:
        ! $runline

/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz
/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz
/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz
/bin/sh: module: comma

++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz
/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz
/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz
/bin/sh: module: command not found
++ 3dcalc: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using pred

In [6]:
# Register MNI152 to NN27
reg_out_file = Path('../great_expectations_data/previous_metaanalysis_data/awpy/anat.un.aff.qw_WARP.nii')
if not reg_out_file.exists():
    ! module load afni; auto_warp.py -base ../great_expectations_data/previous_metaanalysis_data/MNI152_T1_1mm_brain.nii.gz -input ../great_expectations_data/previous_metaanalysis_data/TT_N27+tlrc.HEAD -suffix _toMNI
    ! mv ./awpy ../great_expectations_data/previous_metaanalysis_data/ 

In [7]:
# Transform images to MNI space
!module load afni; 3dNwarpApply -nwarp ../great_expectations_data/previous_metaanalysis_data/awpy/anat.un.aff.qw_WARP.nii ../great_expectations_data/previous_metaanalysis_data/awpy/anat.un.aff.Xat.1D -source ../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz -interp NN -overwrite -master ../great_expectations_data/previous_metaanalysis_data/MNI152_T1_1mm_brain.nii.gz -prefix ../great_expectations_data/previous_metaanalysis_data/ball_pos_mni.nii.gz -overwrite
!module load afni; 3dNwarpApply -nwarp ../great_expectations_data/previous_metaanalysis_data/awpy/anat.un.aff.qw_WARP.nii ../great_expectations_data/previous_metaanalysis_data/awpy/anat.un.aff.Xat.1D -source ../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz -interp NN -overwrite -master ../great_expectations_data/previous_metaanalysis_data/MNI152_T1_1mm_brain.nii.gz -prefix ../great_expectations_data/previous_metaanalysis_data/ball_neg_mni.nii.gz -overwrite

/bin/sh: module: command not found
++ 3dNwarpApply: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: Zhark the Warped
++ opened source dataset '../great_expectations_data/previous_metaanalysis_data/ball_pos.nii.gz'
++ -master dataset is '../great_expectations_data/previous_metaanalysis_data/MNI152_T1_1mm_brain.nii.gz'
++ Processing -nwarp 
++ Warping:.Z
++ Output dataset ../great_expectations_data/previous_metaanalysis_data/ball_pos_mni.nii.gz
++ total CPU time = 3.7 sec  Elapsed = 1.7
/bin/sh: module: command not found
++ 3dNwarpApply: AFNI version=AFNI_19.3.12 (Nov  8 2019) [64-bit]
++ Authored by: Zhark the Warped
++ opened source dataset '../great_expectations_data/previous_metaanalysis_data/ball_neg.nii.gz'
++ -master dataset is '../great_expectations_data/previous_metaanalysis_data/MNI152_T1_1mm_brain.nii.gz'
++ Processing -nwarp 
++ Warping:.Z
++ Output dataset ../great_expectations_data/previous_metaanalysis_data/ball_neg_mni.nii.gz
++ total CPU time = 3.8 sec  

For figure 1A, I loaded `../great_expectations_data/previous_metaanalysis_data/ball_pos_mni.nii.gz` and `../great_expectations_data/previous_metaanalysis_data/ng_pos.nii.gz` into MRICROgl.

For figure 1B, I used `../great_expectations_data/previous_metaanalysis_data/ball_neg_mni.nii.gz`, `../great_expectations_data/previous_metaanalysis_data/ng_neg.nii.gz`, and `../great_expectations_data/previous_metaanalysis_data/ALEtxt_feedback_MDD+HR+ContvsHV_final_woROI_wsmoski2011_wJon_ALE_C05_1_mni152_thresh.nii.gz`